# Group Agent Lab: Tire Specialist and Producer Team

**Objective:** Create a CrewAI-based multi-agent system that helps users find the best tire deals for their car.

**Agents:**
- **Tire Specialist**: Determines the correct tire size for a given car (year, make, model)
- **Tire Producer**: Finds the best tire deals/offers for a specific tire size

**Workflow:** Sequential process where the tire specialist determines the size, then the producer finds the best deals.

## 1. Setup and Imports

In [20]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    print("ERROR: OPENAI_API_KEY not found. Please check your .env file.")
else:
    print("✓ OpenAI API key found")

if not os.getenv("TAVILY_API_KEY"):
    print("WARNING: TAVILY_API_KEY not found. Tire search will be limited without this key.")
else:
    print("✓ Tavily API key found")

✓ OpenAI API key found
✓ Tavily API key found


In [21]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from crewai.tools import BaseTool
from typing import Dict, List

# Use a LangChain model as the LLM for the agents
llm = ChatOpenAI(model="gpt-4o")

## 2. Define the Agents

Create two specialized agents following the CrewAI pattern, just like in the reference setup.

In [22]:
# Agent 1: Tire Specialist
tire_specialist = Agent(
    role='Automotive Tire Specialist',
    goal='Determine the correct tire size for any vehicle based on year, make, and model.',
    backstory=(
        'You are an expert automotive technician with 15 years of experience in tire '
        'sizing and vehicle specifications. You have comprehensive knowledge of OEM '
        'tire specifications for all major vehicle manufacturers and can determine '
        'the exact tire size that a vehicle requires based on its year, make, and model.'
    ),
    verbose=True,
    llm=llm
)

# Agent 2: Tire Producer/Deal Finder
tire_producer = Agent(
    role='Tire Deal Specialist',
    goal='Find the best tire deals and offers for a specific tire size.',
    backstory=(
        'You are a procurement expert who specializes in finding the best tire deals '
        'across multiple retailers. You have access to pricing databases and can '
        'quickly identify the most cost-effective options while considering quality '
        'and customer ratings.'
    ),
    verbose=True,
    llm=llm
)

print("✓ Agents created successfully")

✓ Agents created successfully


## 3. Adding Tools to Agents

Let's create a custom tool that can search for real tire deals using the Tavily search API, similar to how we added search tools in the reference setup. This tool will find actual current pricing from tire retailers.

In [23]:
from tavily import TavilyClient

class TireDealSearchTool(BaseTool):
    name: str = "TireDealSearch"
    description: str = "A tool that searches for current tire deals and prices from various retailers for a specific tire size."
    
    def _run(self, tire_size: str) -> str:
        """
        Search for actual tire deals using Tavily search API.
        This will find current pricing and availability from real retailers.
        """
        if not os.getenv("TAVILY_API_KEY"):
            return f"TAVILY_API_KEY not found. Please set up your API key to search for tire deals for size {tire_size}"
        
        try:
            client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
            
            # Create a specific search query for tire deals
            query = f"tire deals {tire_size} price comparison TireRack Discount Tire Costco current prices"
            
            # Search for tire deals
            response = client.search(query=query, search_depth="basic", max_results=5)
            
            if not response.get('results'):
                return f"No tire deals found for size {tire_size}. Please try a different tire size."
            
            # Format the search results
            result = f"Current tire deals found for size {tire_size}:\n\n"
            
            for i, item in enumerate(response['results'][:3], 1):  # Limit to top 3 results
                title = item.get('title', 'No title')
                url = item.get('url', 'No URL')
                content = item.get('content', 'No description available')
                
                # Truncate content if too long
                if len(content) > 200:
                    content = content[:200] + "..."
                
                result += f"{i}. **{title}**\n"
                result += f"   Source: {url}\n"
                result += f"   Details: {content}\n\n"
            
            result += "💡 **Tip:** Visit these retailer websites directly for the most current pricing and availability."
            
            return result
            
        except Exception as e:
            return f"Error searching for tire deals: {str(e)}. Please check your TAVILY_API_KEY and try again."

# Instantiate the tool
tire_search_tool = TireDealSearchTool()

# Now assign the tool to the tire producer agent
tire_producer.tools = [tire_search_tool]
print("✓ Real tire search tool created and assigned to Tire Producer agent")

✓ Real tire search tool created and assigned to Tire Producer agent


## 4. Define Tasks

Create tasks that define what each agent should do, using the agents we defined earlier.

## 5. Create and Execute the Crew

Now let's put it all together in a function that creates and runs the crew with our agents.

In [24]:
def create_tasks(year: int, make: str, model: str):
    """
    Create tasks dynamically based on the car information provided.
    """
    
    # Task 1: Determine tire size
    tire_size_task = Task(
        description=(
            f'Determine the correct OEM tire size for a {year} {make} {model}. '
            f'Provide ONLY the tire size in the standard format (e.g., "215/55R17") '
            f'with no additional text or explanation.'
        ),
        expected_output='A single tire size string in the format XXX/XXRXX (e.g., "215/55R17")',
        agent=tire_specialist
    )
    
    # Task 2: Find tire deals (depends on the output of Task 1)
    tire_deals_task = Task(
        description=(
            'Using the tire size determined by the tire specialist, find the top 3 '
            'best tire deals. Use the TireDealSearch tool to get current pricing '
            'and present the results in a clear, organized format with retailer, '
            'brand, price, and rating information.'
        ),
        expected_output=(
            'A formatted list of the top 3 tire deals including retailer name, '
            'tire brand, price, and customer rating for each option.'
        ),
        agent=tire_producer,
        context=[tire_size_task]  # This task depends on the output of tire_size_task
    )
    
    return tire_size_task, tire_deals_task

print("✓ Task creation function ready")

✓ Task creation function ready


## 6. Example Usage

Let's test our tire finding crew with some example cars.

In [ ]:
def find_tire_deals(year: int, make: str, model: str):
    """
    Main function to find tire deals for a specific car.
    """
    print(f"🚗 Finding tire deals for: {year} {make} {model}")
    print("=" * 50)
    
    # Create tasks for this specific car
    tire_size_task, tire_deals_task = create_tasks(year, make, model) #expose
    
    # Create the crew with sequential process
    tire_crew = Crew(
        agents=[tire_specialist, tire_producer],
        tasks=[tire_size_task, tire_deals_task],
        process=Process.sequential,
        verbose=True
    )
    
    # Execute the crew
    result = tire_crew.kickoff() #endpoint to expose
    
    print("\n" + "=" * 50)
    print("🎯 FINAL RESULTS")
    print("=" * 50)
    print(result)
    
    return result

print("✓ Main function ready")

✓ Main function ready


## 6. Example Usage

Let's test our tire finding crew with some example cars.

In [26]:
# Example 1: Toyota Camry
result1 = find_tire_deals(2020, "Toyota", "Camry")

🚗 Finding tire deals for: 2020 Toyota Camry


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e6a4f71f-0229-4445-a670-6c464bcda71e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Automotive Tire Specialist                                                                              │
│                                                                                                                 │
│  Task: Determine the correct OEM tire size for a 2020 Toyota Camry. Provide ONLY the tire size in the standard  │
│  format (e.g., "215/55R17") with no additional text or explanation.                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Automotive Tire Specialist                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  235/45R18                                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 06a951d2-ba14-46dd-8991-0a3a8cb80dc4                                                                     │
│  Agent: Automotive Tire Specialist                                                                              │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tire Deal Specialist                                                                                    │
│                                                                                                                 │
│  Thought: Thought: I need to search for the current tire deals and prices from various retailers for the tire   │
│  size 235/45R18.                                                                                                │
│                                                                                                                 │
│  Using Tool: TireDealSearch                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tire Deal Specialist                                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. **Tire Rack**                                                                                               │
│     - Retailer: Tire Rack                                                                                       │
│     - Brand: Various                                                                                            │
│     - Price: Set of 4 for $820.00                                                                               │
│     - Customer Rating: Not provided                                                                             │
│     - Additional Info: Includes free road hazard protection and two-year coverage with a 60K mile               │
│  manufacturer's warranty.                                                                                       │
│                                                                                                                 │
│  2. **Walmart**                                                                                                 │
│     - Retailer: Walmart.com                                                                                     │
│     - Brand: Various                                                                                            │
│     - Price: Starting at $83.93 each                                                                            │
│     - Customer Rating: Example ratings from listings (344 reviews, 83 rating, etc.)                             │
│     - Additional Info: Prices vary based on specific tire choice, with different options available.             │
│                                                                                                                 │
│  3. **Discount Tire**                                                                                           │
│     - Retailer: Discount Tire                                                                                   │
│     - Brand: Various                                                                                            │
│     - Price: Available upon inquiry at their website                                                            │
│     - Customer Rating: Not specified, varies by brand                                                           │
│     - Additional Info: Wide selection available with potential promos directly on the website or at local       │
│  stores.                                                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b5e5a08d-f173-4b4b-8ca2-fa8f5b8bcaa4                                                                     │
│  Agent: Tire Deal Specialist                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [27]:
# Example 2: Honda Civic
result2 = find_tire_deals(2019, "Honda", "Civic")

🚗 Finding tire deals for: 2019 Honda Civic


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: cc8ab540-142c-4217-94b4-1c06332acbe2                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Automotive Tire Specialist                                                                              │
│                                                                                                                 │
│  Task: Determine the correct OEM tire size for a 2019 Honda Civic. Provide ONLY the tire size in the standard   │
│  format (e.g., "215/55R17") with no additional text or explanation.                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Automotive Tire Specialist                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  215/55R16                                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tire Deal Specialist                                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Here's a formatted list of the top 3 tire deals for the size 215/55R16 including retailer name, tire brand,    │
│  price, and customer rating where available:                                                                    │
│                                                                                                                 │
│  1. **Retailer:** Tire Rack                                                                                     │
│     - **Brand:** Unspecified                                                                                    │
│     - **Price:** $196.58 per tire                                                                               │
│     - **Rating:** Unspecified                                                                                   │
│     - **Details:** Free Road Hazard Protection valued at $19.29, Two-year coverage, 60K Mile Manufacturer's     │
│  Warranty.                                                                                                      │
│                                                                                                                 │
│  2. **Retailer:** Costco / Discount Tire Price Match                                                            │
│     - **Brand:** Michelin CrossClimate2                                                                         │
│     - **Price:** Unspecified, matched Costco price                                                              │
│     - **Rating:** Unspecified                                                                                   │
│     - **Details:** 215/55R16/XL, Speed Rating: H, Load Index: 97, Mud feature.                                  │
│                                                                                                                 │
│  3. **Retailer:** Discount Tire                                                                                 │
│     - **Brand:** Various brands available                                                                       │
│     - **Price:** Unspecified                                                                                    │
│     - **Rating:** Unspecified                                                                                   │
│     - **Details:** Selection available based on tire width, aspect ratio, and rim size across various brands    │
│  and fitments.                                                                                                  │
│                                                                                                                 │
│  💡 Note: For detailed pricing and customer ratings, visit the retailer websites directly as prices and         │
│  availability can change frequently.                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4499b99b-af9d-4166-9418-7b75eb31f949                                                                     │
│  Agent: Tire Deal Specialist                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


🎯 FINAL RESULTS
Here's a formatted list of the top 3 tire deals for the size 215/55R16 including retailer name, tire brand, price, and customer rating where available:

1. **Retailer:** Tire Rack
   - **Brand:** Unspecified
   - **Price:** $196.58 per tire
   - **Rating:** Unspecified
   - **Details:** Free Road Hazard Protection valued at $19.29, Two-year coverage, 60K Mile Manufacturer's Warranty.

2. **Retailer:** Costco / Discount Tire Price Match
   - **Brand:** Michelin CrossClimate2
   - **Price:** Unspecified, matched Costco price
   - **Rating:** Unspecified
   - **Details:** 215/55R16/XL, Speed Rating: H, Load Index: 97, Mud feature.

3. **Retailer:** Discount Tire
   - **Brand:** Various brands available
   - **Price:** Unspecified
   - **Rating:** Unspecified
   - **Details:** Selection available based on tire width, aspect ratio, and rim size across various brands and fitments.

💡 Note: For detailed pricing and customer ratings, visit the retailer websites directly as pri

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: cc8ab540-142c-4217-94b4-1c06332acbe2                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Here's a formatted list of the top 3 tire deals for the size 215/55R16 including retailer name,  │
│  tire brand, price, and customer rating where available:                                                        │
│                                                                                                                 │
│  1. **Retailer:** Tire Rack                                                                                     │
│     - **Brand:** Unspecified                                                                                    │
│     - **Price:** $196.58 per tire                                                                               │
│     - **Rating:** Unspecified                                                                                   │
│     - **Details:** Free Road Hazard Protection valued at $19.29, Two-year coverage, 60K Mile Manufacturer's     │
│  Warranty.                                                                                                      │
│                                                                                                                 │
│  2. **Retailer:** Costco / Discount Tire Price Match                                                            │
│     - **Brand:** Michelin CrossClimate2                                                                         │
│     - **Price:** Unspecified, matched Costco price                                                              │
│     - **Rating:** Unspecified                                                                                   │
│     - **Details:** 215/55R16/XL, Speed Rating: H, Load Index: 97, Mud feature.                                  │
│                                                                                                                 │
│  3. **Retailer:** Discount Tire                                                                                 │
│     - **Brand:** Various brands available                                                                       │
│     - **Price:** Unspecified                                                                                    │
│     - **Rating:** Unspecified                                                                                   │
│     - **Details:** Selection available based on tire width, aspect ratio, and rim size across various brands    │
│  and fitments.                                                                                                  │
│                                                                                                                 │
│  💡 Note: For detailed pricing and customer ratings, visit the retailer websites directly as prices and         │
│  availability can change frequently.                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# Example 3: Ford F-150
result3 = find_tire_deals(2021, "Ford", "F-150")

## 7. Interactive Demo

Create an interactive version where users can input their own car details.

In [ ]:
def interactive_tire_finder():
    """
    Interactive function for users to input their car details.
    """
    print("🔧 Interactive Tire Finder")
    print("Enter your car details to find the best tire deals:\n")
    
    try:
        year = int(input("Enter the year of your car: "))
        make = input("Enter the make of your car (e.g., Toyota, Honda): ").strip()
        model = input("Enter the model of your car (e.g., Camry, Civic): ").strip()
        
        if year < 1990 or year > 2025:
            print("⚠️ Please enter a year between 1990 and 2025")
            return
        
        if not make or not model:
            print("⚠️ Please enter both make and model")
            return
        
        # Run the tire finding crew
        result = find_tire_deals(year, make, model)
        return result
        
    except ValueError:
        print("⚠️ Please enter a valid year (numeric value)")
    except KeyboardInterrupt:
        print("\n👋 Goodbye!")
    except Exception as e:
        print(f"❌ An error occurred: {str(e)}")

# Uncomment the line below to run the interactive demo
interactive_tire_finder()

## Lab Summary

In this lab, you've successfully created a CrewAI-based multi-agent system that:

**✅ What we accomplished:**
- Created two specialized agents with distinct roles and expertise
- Implemented a custom tool for tire deal searching
- Set up a sequential workflow where agents collaborate
- Used context passing between tasks to maintain workflow dependencies
- Provided both automated examples and interactive functionality

**🔧 Key CrewAI Concepts Applied:**
- **Agent**: Role-based AI entities with specific goals and backstories
- **Task**: Defined work units with clear descriptions and expected outputs
- **Crew**: Orchestration of agents and tasks with defined processes
- **Tools**: Custom capabilities that extend agent functionality
- **Context**: Task dependencies that allow information flow between agents

**🚀 Next Steps:**
- Replace the simulated tire search tool with real API integrations
- Add more sophisticated error handling and validation
- Implement additional agents (e.g., installation service finder, warranty specialist)
- Add hierarchical process for more complex workflows
- Integrate with external databases for more accurate tire specifications